<a href="https://colab.research.google.com/github/miladramzy/MCDM_DOE_UBC_ENGR_589/blob/main/Lab5_MCDM_MADM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENGR 489/589
### School of Engineering - Okanagan Campus
### By Milad Ramezankhani, Dr. Abbas Milani
#### milad.ramezankhani@ubc.ca | https://miladramzy.github.io/
## Lab 5 - MADM


## Approach 1 - Manual calculations

In [1]:
## import libraries
import pandas as pd
import numpy as np

In [2]:
## data from chapter 5 - TOPSIS example
mtx = [[690,3.1,9,7,4]
      ,[590,3.9,7,6,10]
      ,[600,3.6,8,8,7]
      ,[620,3.8,7,10,6]
      ,[700,2.8,10,4,6]
      ,[650,4.0,6,9,8]]

In [3]:
## weight
w = [0.3, 0.2,0.2, 0.15, 0.15]

In [4]:
## Altenative names
Names = ['Alfred', 'Beverly', 'Calvin', 'Diane', 'Edward', 'Fran']

In [5]:
## Create dataframe
Data = pd.DataFrame(mtx, index = Names)
Data

,0,1,2,3,4
Alfred,690,3.1,9,7,4
Beverly,590,3.9,7,6,10
Calvin,600,3.6,8,8,7
Diane,620,3.8,7,10,6
Edward,700,2.8,10,4,6
Fran,650,4.0,6,9,8


In [6]:
## STEP 1 - Normalization
Data_norm = Data/np.sqrt(np.power(Data,2).sum(axis=0))

In [7]:
Data_norm

,0,1,2,3,4
Alfred,0.438053,0.355454,0.462299,0.376322,0.230556
Beverly,0.374567,0.447184,0.359566,0.322562,0.576390
Calvin,0.380916,0.412785,0.410932,0.430083,0.403473
Diane,0.393613,0.435718,0.359566,0.537603,0.345834
Edward,0.444402,0.321055,0.513665,0.215041,0.345834
Fran,0.412659,0.458650,0.308199,0.483843,0.461112


In [8]:
## STEP 2 - Weighted normalized ratings
Data_norm_w = Data_norm*w
Data_norm_w

,0,1,2,3,4
Alfred,0.131416,0.071091,0.092460,0.056448,0.034583
Beverly,0.112370,0.089437,0.071913,0.048384,0.086459
Calvin,0.114275,0.082557,0.082186,0.064512,0.060521
Diane,0.118084,0.087144,0.071913,0.080640,0.051875
Edward,0.133321,0.064211,0.102733,0.032256,0.051875
Fran,0.123798,0.091730,0.061640,0.072576,0.069167


In [10]:
## STEP 3 - Identifying positive and negative ideals
positive_ideal = Data_norm_w.max()
negative_ideal = Data_norm_w.min()

In [12]:
Data_norm_w.max()

0    0.133321
1    0.091730
2    0.102733
3    0.080640
4    0.086459
dtype: float64

In [13]:
## STEP 4 - Seperation Measurements:
##Positive Ideal
SM_P = np.sqrt(np.power(Data_norm_w-positive_ideal,2).sum(axis=1))
##Negative Ideal
SM_N = np.sqrt(np.power(Data_norm_w-negative_ideal,2).sum(axis=1))

In [14]:
np.sqrt(np.power(Data_norm_w-positive_ideal,2).sum(axis=1))

Alfred     0.061737
Beverly    0.049341
Calvin     0.042449
Diane      0.048980
Edward     0.065531
Fran       0.046297
dtype: float64

In [15]:
SM_N/(SM_N+SM_P)

Alfred     0.416704
Beverly    0.551900
Calvin     0.539620
Diane      0.539926
Edward     0.429128
Fran       0.568142
dtype: float64

## Approach 2 - sk-criteria library

In [18]:
pip install scikit-criteria
import pandas as pd
import numpy as np
import skcriteria as skc

In [19]:
## data from chapter 5 - TOPSIS example
mtx = [[690,3.1,9,7,4]
      ,[590,3.9,7,6,10]
      ,[600,3.6,8,8,7]
      ,[620,3.8,7,10,6]
      ,[700,2.8,10,4,6]
      ,[650,4.0,6,9,8]]

## weights
w = [0.3, 0.2,0.2, 0.15, 0.15]

## Alternatives
alternatives = ['Alfred', 'Beverly', 'Calvin', 'Diane', 'Edward', 'Fran']

## Attributes
criteria = ['GRE', 'GPA', 'college', 'recommendation', 'interview']

## Identifying benefit (MAX) and cost (MIN) attributes
objectives = [max, max, max, max, max]

dm = skc.mkdm(mtx, objectives, weights = w, alternatives = alternatives, criteria=criteria)

In [ ]:
dm.plot.wheatmap()

In [ ]:
dm.plot.wbar(cmap="viridis")

In [22]:
from skcriteria.preprocessing import invert_objectives, scalers # Preporcessing steps
from skcriteria.madm import similarity  # here lives TOPSIS
from skcriteria.pipeline import mkpipe  # this function is for create pipelines

In [23]:
# TOPSIS
# Create a pipeline to perform TOPSIS
pipe = mkpipe(
    invert_objectives.InvertMinimize(), # Convert cost attributes to benefit attributes
    scalers.VectorScaler(target="matrix"),  # this scaler transform the matrix. The other option is SumScaler.
    similarity.TOPSIS(), # TOPSIS method
)

# Evaluation
rank = pipe.evaluate(dm)
rank

Alternatives,Alfred,Beverly,Calvin,Diane,Edward,Fran
Rank,6,2,4,3,5,1


In [24]:
print(rank.e_)
print("Ideal:", rank.e_.ideal)
print("Anti-Ideal:", rank.e_.anti_ideal)
print("Similarity index:", rank.e_.similarity)

<extra {'ideal', 'anti_ideal', 'similarity'}>
Ideal: [0.13332057 0.09173009 0.1027331  0.0806405  0.08645856]
Anti-Ideal: [0.11237019 0.06421106 0.06163986 0.0322562  0.03458343]
Similarity index: [0.41670356 0.55190009 0.53961966 0.53992627 0.42912804 0.56814199]


In [ ]:
# ELECTRE
from skcriteria.madm import electre

pipe_vector = mkpipe(
    invert_objectives.InvertMinimize(),
    scalers.VectorScaler(target="matrix"),  # this scaler transform the matrix
    electre.ELECTRE1(p=0.65, q=0.35),
)

kernel_vector = pipe_vector.evaluate(dm)
kernel_vector

In [ ]:
# WSM
from skcriteria.madm import simple
dec = simple.WeightedSumModel()

inverter = invert_objectives.InvertMinimize()
dmt = inverter.transform(dm)

scaler = scalers.SumScaler(target="both")
dmt = scaler.transform(dmt)

rank = dec.evaluate(dmt)  # we use the tansformed version of the data
rank